In [1]:
df = spark.read.format("csv").option("header","true").load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "abfss://dlta@onelake.dfs.fabric.microsoft.com/hi.Lakehouse/Files/products/products.csv".
display(df)

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1e29558a-4766-4001-ae5d-0bcd153becf8)

In [2]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 4, Finished, Available)

In [5]:
#storing metadata in metastore but data in external table
df.write.format("delta").saveAsTable("external_products", path="abfss://dlta@onelake.dfs.fabric.microsoft.com/hi.Lakehouse/Files/external_products")

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 7, Finished, Available)

In [6]:
%%sql

DESCRIBE FORMATTED managed_products;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 8, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [7]:
%%sql

DESCRIBE FORMATTED external_products;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 9, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [8]:
%%sql

DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
df.write.format("delta").saveAsTable("external_products", path="abfss://dlta@onelake.dfs.fabric.microsoft.com/hi.Lakehouse/Files/external_products1")

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 14, Finished, Available)

In [13]:
%%sql
DROP TABLE external_products;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 16, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
%%sql

CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 17, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
%%sql

SELECT * FROM products LIMIT 10 ;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 18, Finished, Available)

<Spark SQL result set with 10 rows and 4 fields>

In [16]:
%%sql

UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 19, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [17]:
%%sql

DESCRIBE HISTORY products;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 20, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [18]:
delta_table_path = 'Files/external_products'

# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, 188f4681-a0d0-4ad0-a1a6-e337f796fe42)

SynapseWidget(Synapse.DataFrame, 1a92f7f4-a270-40dd-ae23-fdaebcf90e4a)

In [19]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''
mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
print("Source stream created...")

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 22, Finished, Available)

Source stream created...


In [20]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 23, Finished, Available)

Streaming to delta sink...


In [21]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 24, Finished, Available)

<Spark SQL result set with 9 rows and 2 fields>

In [22]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 25, Finished, Available)

True

In [23]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 26, Finished, Available)

<Spark SQL result set with 16 rows and 2 fields>

In [24]:
deltastream.stop()

StatementMeta(, 17dc32e9-e59f-4025-ad73-8c0a28e37d46, 27, Finished, Available)